## SQS for survey distribution



Summer Negahdar


In [34]:
## importing packages

import boto3
import json
import time

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
aws_lambda = boto3.client('lambda', region_name='us-east-1')
sqs = boto3.client('sqs', region_name= "us-east-1")
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

In [36]:
# Step 2: Create DynamoDB Table
table = dynamodb.create_table(
    TableName='SurveyResponses_Summer99D_HW06',
    KeySchema=[
        {'AttributeName': 'user_id', 'KeyType': 'HASH'},
        {'AttributeName': 'timestamp', 'KeyType': 'RANGE'}
    ],
    AttributeDefinitions=[
        {'AttributeName': 'user_id', 'AttributeType': 'S'},
        {'AttributeName': 'timestamp', 'AttributeType': 'S'}
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)
table.meta.client.get_waiter('table_exists').wait(TableName='SurveyResponses_Summer99D_HW06')
print("DynamoDB table SurveyResponses_Summer99D_HW06 created.")


DynamoDB table SurveyResponses_Summer99D_HW06 created.


In [37]:
# Step 3: Get IAM Role ARN
role = iam_client.get_role(RoleName='LabRole')
role_arn = role['Role']['Arn']
print("Retrieved LabRole ARN.")



Retrieved LabRole ARN.


In [40]:

# Step 4: Create Lambda Function 
with open('HW6.zip', 'rb') as f:
    lambda_zip = f.read()
aws_lambda.create_function(
    FunctionName='HW6',
    Runtime='python3.9',
    Role=role_arn,
    Handler='lambda_function.lambda_handler',
    Code=dict(ZipFile=lambda_zip),
    Timeout=30
)
print("Lambda function HW6 created.")

Lambda function HW6 created.


In [44]:
# Step 3: Create SQS Queue
queue_url = sqs.create_queue(QueueName='survey_queue_summer99d')['QueueUrl']
print(f"SQS queue created: {queue_url}")

SQS queue created: https://sqs.us-east-1.amazonaws.com/544835564974/survey_queue_summer99d


In [45]:
# Step 6: Get SQS Queue ARN
sqs_info = sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])
sqs_arn = sqs_info['Attributes']['QueueArn']
print(f"SQS ARN: {sqs_arn}")

SQS ARN: arn:aws:sqs:us-east-1:544835564974:survey_queue_summer99d


In [47]:
# Step 6: Connect SQS to Lambda
aws_lambda.create_event_source_mapping(
    EventSourceArn=sqs_arn,
    FunctionName='HW6',
    Enabled=True,
    BatchSize=10
)
print("SQS queue linked to Lambda function.")

print(f"SQS Queue URL: {queue_url}")

SQS queue linked to Lambda function.
SQS Queue URL: https://sqs.us-east-1.amazonaws.com/544835564974/survey_queue_summer99d


In [48]:
def send_survey(survey_path, sqs_url):
    sqs = boto3.client('sqs', region_name='us-east-1')
    
    # Read the JSON survey file
    with open(survey_path, 'r') as f:
        survey_data = json.load(f)
    
    # Convert the survey data to a string
    survey_str = json.dumps(survey_data)
    
    # Send the survey data to the SQS queue
    response = sqs.send_message(
        QueueUrl=sqs_url,
        MessageBody=survey_str
    )
    
    # Check if the message was sent successfully
    if response.get('MessageId'):
        return 200
    return 400

In [49]:


# List of survey files
survey_files = [
    'tests/test1.json',
    'tests/test2.json',
    'tests/test3.json',
    'tests/test4.json',
    'tests/test5.json',
    'tests/test6.json',
    'tests/test7.json',
    'tests/test8.json',
    'tests/test9.json',
    'tests/test10.json'
]

sqs_url = "https://sqs.us-east-1.amazonaws.com/544835564974/survey_queue_summer99d"

for survey_file in survey_files:
    status = send_survey(survey_file, sqs_url)
    print(status)
    time.sleep(10)


200
200
200
200
200
200
200
200
200
200


In [52]:

# Verify DynamoDB table
print("\nDynamoDB table contents:")
table = dynamodb.Table("SurveyResponses_Summer99D_HW06")
records = table.scan()['Items']
if records:
    for record in records:
        print(record)
else:
    print("No records found in SurveyResponses_Summer99D_HW06 table. Checking table status...")
    dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')
    try:
        table_description = dynamodb_client.describe_table(TableName="SurveyResponses_Summer99D_HW06")
        print(f"Table status: {table_description['Table']['TableStatus']}")
        print(f"Table item count: {table_description['Table']['ItemCount']}")
    except dynamodb_client.exceptions.ResourceNotFoundException:
        print("Table SurveyResponses_Summer99D_HW06 does not exist.")


DynamoDB table contents:
{'q1': Decimal('3'), 'q2': Decimal('3'), 'user_id': '0005', 'q3': Decimal('3'), 'q4': Decimal('3'), 'q5': Decimal('3'), 'num_submission': Decimal('1'), 'timestamp': '092821122000', 'freetext': "I'm feeling okay, but not spectacular"}
{'q1': Decimal('5'), 'q2': Decimal('3'), 'user_id': '0001', 'q3': Decimal('2'), 'q4': Decimal('2'), 'q5': Decimal('4'), 'num_submission': Decimal('5'), 'timestamp': '092821120000', 'freetext': 'I had a very bad day today...'}
{'q1': Decimal('5'), 'q2': Decimal('3'), 'user_id': '0001', 'q3': Decimal('2'), 'q4': Decimal('2'), 'q5': Decimal('4'), 'num_submission': Decimal('2'), 'timestamp': '092821120100', 'freetext': ''}
{'q1': Decimal('1'), 'q2': Decimal('3'), 'user_id': '0001', 'q3': Decimal('2'), 'q4': Decimal('2'), 'q5': Decimal('5'), 'num_submission': Decimal('3'), 'timestamp': '092921120000', 'freetext': 'Not looking forward to going to work today'}
{'q1': Decimal('1'), 'q2': Decimal('1'), 'user_id': '0001', 'q3': Decimal('2')

In [53]:
# Verify S3 bucket
print("S3 bucket contents:")
response = s3_client.list_objects_v2(Bucket='hw05-summer99d')
objects = response.get('Contents', [])
if objects:
    for obj in objects:
        print(obj['Key'])
else:
    print("No objects found in S3 bucket hw05-summer99d.")

S3 bucket contents:
0001092821120000.json
0001092821120100.json
0001092921120000.json
0001093021120300.json
0002092821120000.json
0003092821120001.json
0004092821120002.json
0005092821122000.json
test_responses/response_0.json
test_responses/response_1.json
test_responses/response_2.json
test_responses/response_3.json
test_responses/response_4.json
test_responses/response_5.json
test_responses/response_6.json


In [56]:
# Delete each pipeline component if it still exists:
# Lambda
try:
    aws_lambda.delete_function(FunctionName='HW6')
    print("Lambda Function Deleted")
except aws_lambda.exceptions.ResourceNotFoundException:
    print("AWS Lambda Function Already Deleted")

# SQS
try:
    sqs.delete_queue(QueueUrl=sqs_url)
    print("SQS Queue Deleted")
except sqs.exceptions.QueueDoesNotExist:
    print("SQS Queue Already Deleted")

# DynamoDB
dynamodb = boto3.client('dynamodb', region_name= 'us-east-1')
try:
    response = dynamodb.delete_table(TableName='SurveyResponses_Summer99D_HW06')
    print("DynamoDB Table Deleted")
except dynamodb.exceptions.ResourceNotFoundException:
    print("DynamoDB Table Already Deleted")

AWS Lambda Function Already Deleted
SQS Queue Already Deleted
DynamoDB Table Deleted
